In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train = train.drop(['id', 'hour_bef_ozone'], axis=1)
test = test.drop(['id', 'hour_bef_ozone'], axis=1)

In [5]:
y = train['count']
x = train.drop('count', axis=1)

In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [7]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y).reshape(-1, 1)
test_tensor = torch.FloatTensor(test)

In [8]:
layer1 = torch.nn.Linear(x_tensor.shape[1], 16)
layer2 = torch.nn.Linear(16, 1)

ReLU = torch.nn.ReLU()

model = torch.nn.Sequential(layer1, ReLU, layer2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

loss = torch.nn.MSELoss()

In [9]:
epochs = 15000

for epoch in range(epochs):
    optimizer.zero_grad()
    hypothesis = model(x_tensor)
    cost = loss(hypothesis, y_tensor)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'{epoch}/{epochs} cost: {cost.item()}')

0/15000 cost: 19198.76953125
100/15000 cost: 2736.331298828125
200/15000 cost: 2691.0205078125
300/15000 cost: 2663.20361328125
400/15000 cost: 2641.199951171875
500/15000 cost: 2604.494384765625
600/15000 cost: 2546.86767578125
700/15000 cost: 2451.29248046875
800/15000 cost: 2351.98779296875
900/15000 cost: 2276.983642578125
1000/15000 cost: 2231.03466796875
1100/15000 cost: 2202.695068359375
1200/15000 cost: 2175.55810546875
1300/15000 cost: 2150.65087890625
1400/15000 cost: 2127.733642578125
1500/15000 cost: 2107.16064453125
1600/15000 cost: 2084.385009765625
1700/15000 cost: 2065.61328125
1800/15000 cost: 2047.619384765625
1900/15000 cost: 2033.6729736328125
2000/15000 cost: 2022.2330322265625
2100/15000 cost: 2012.8499755859375
2200/15000 cost: 2003.2125244140625
2300/15000 cost: 1992.340087890625
2400/15000 cost: 1983.671630859375
2500/15000 cost: 1974.7825927734375
2600/15000 cost: 1968.70556640625
2700/15000 cost: 1964.3294677734375
2800/15000 cost: 1960.2940673828125
2900/150

In [10]:
predict = model(test_tensor)
submit = pd.read_csv('submit_sample.csv')
submit['count'] = predict.detach()
submit.to_csv('submit.csv', index=False)